In [16]:
from myenv.models.candlestick import Candlestick
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


candlestick = Candlestick()

bitcoin_prices = candlestick.to_df()

bitcoin_prices.describe()

data = [0]

for index, row in bitcoin_prices.iterrows():
    if index < 1 or index > bitcoin_prices['close'].count() - 1:
        continue

    dkm = 0
    if row['low'] < bitcoin_prices.iloc[index - 1]['low'] or row['high'] > bitcoin_prices.iloc[index - 1]['high']:
        dkm = 1

    data.append(dkm)

bitcoin_prices['dkm'] = data
print(bitcoin_prices)


           date      open      high     close       low  dkm
0    2017-08-17   4261.48   4485.39   4285.08   4200.74    0
1    2017-08-18   4285.08   4371.52   4108.37   3938.77    1
2    2017-08-19   4108.37   4184.69   4139.98   3850.00    1
3    2017-08-20   4120.98   4211.08   4086.29   4032.62    1
4    2017-08-21   4069.13   4119.62   4016.00   3911.79    1
...         ...       ...       ...       ...       ...  ...
1417 2021-07-04  34669.10  35967.90  35286.50  34357.10    1
1418 2021-07-05  35288.10  35293.80  33690.10  33125.60    1
1419 2021-07-06  33690.10  35118.90  34220.00  33532.00    0
1420 2021-07-07  34220.00  35059.10  33862.10  33777.80    0
1421 2021-07-08  33862.10  33929.60  32834.20  32077.00    1

[1422 rows x 6 columns]
